## Preparación del entorno de trabajo

In [ ]:
# Instalar el módulo con la fórmula de Haversine
!pip install haversine

In [ ]:
# Importar las librerias y módulos necesarios
import pandas as pd # Manejo de datos numéricos
import geopandas as gpd # Manejo de datos espaciales
import fiona # Lectura de archivos vectoriales
from haversine import haversine # Cálculo de la distancia entre coordenadas
import folium # Creación de mapas interactivos

In [ ]:
# Habilitar la lectura de archivos KML
fiona.drvsupport.supported_drivers['KML'] = 'rw'

## Selección de la información de interés del Censo 2020

In [ ]:
# Índices de las series de datos de interés
series = [0, 1, 2, 183, 185, 195, 199]

# Orden propuesto para las series de datos
orden = [0, 2, 1, 3, 4, 5, 6]

In [ ]:
# Lectura de los datos numéricos de interés de todas las colonias en Jalisco
df = pd.read_csv('/content/drive/MyDrive/TECHO/censo-poblacion-vivienda-2020-inegi-informacion-demografica-y-social-colonias-estado-jalisco.csv', usecols=series, encoding='latin-1')

# Lectura de los datos espaciales de todas las colonias en Jalisco
gdf = gpd.read_file('/content/drive/MyDrive/TECHO/censo-poblacion-vivienda-2020-inegi-informacion-demografica-y-social-colonias-estado-jalisco.kml')

# Unión y organización de los datos de todas las colonias en Jalisco
colonias = gpd.GeoDataFrame(df.iloc[:, orden], geometry=gdf.geometry)

## Selección de las colonias cercanas a la oficina de TECHO

In [ ]:
# Ubicación y distancia aceptable a la oficina de TECHO
oficina = [20.7009, -103.3676]
distancia = 20000 # 20 km

In [ ]:
# Por cada colonia
for i, poligono in colonias['geometry'].items():

  # Calcular la distancia entre su centro geométrico y la oficina
  distancia_haversine = haversine([poligono.centroid.y, poligono.centroid.x], oficina)

  # Borrar la información de las colonias lejanas
  if distancia_haversine > distancia/1000:
    colonias.drop(index=i, inplace=True)

## Selección de los municipios cercanos

In [ ]:
# Nombre y número de los municipios dentro de la distancia aceptable
num_municipios = {'Guadalajara': 38,
                  'San Pedro Tlaquepaque': 96,
                  'Zapopan': 118,
                  'Tonal': 99,
                  'El Salto': 68,
                  'Zapotlanejo': 122,
                  'Ixtlahuacn del Ro': 44,
                  'Tlajomulco de Ziga': 95}

In [ ]:
# Definir los nombres de los municipios cercanos
df = pd.DataFrame({'Nombres': list(num_municipios.keys())})

# Lectura de los datos espaciales de los municipios cercanos
gdf = gpd.read_file('/content/drive/MyDrive/TECHO/limitemunicipal_mgj2012_modificadodecreto26837.kml')

# Unión de los datos de los municipios cercanos
municipios = gpd.GeoDataFrame(df, geometry=gdf.iloc[list(num_municipios.values())].geometry.values)

## Selección del municipio de cada colonia cercana

In [ ]:
# Por cada colonia cercana
for i_col, f_col in colonias.iterrows():

  # Selecionar sus datos espaciales
  colonia = f_col[-1]

  # Tupla para guardar el municipio al que pertenece la colonia
  municipio = (0, 0)

  # Por cada municipio
  for i_mun, f_mun in municipios.iterrows():

    # Calcular la intersección entre la colonia y el municipio
    interseccion = colonia.intersection(f_mun[-1]).area

    # Guardar el municipio con la mayor área de intersección
    if interseccion > municipio[1]:
      municipio = (i_mun, interseccion)

  # Actualizar el municipio al que pertenece la colonia
  colonias.at[i_col, colonias.columns[2]] = municipios.iloc[municipio[0]]['Nombres']

In [ ]:
# Exportar la información de las colonias cercanas a un archivo CSV
colonias.iloc[:, 1:7].to_csv('/content/drive/MyDrive/TECHO/InformacionCenso2020/ColoniasCercanasCenso2020.csv')

## Creación de la capa 'Oficina'

In [ ]:
# Crear una capa con información respecto a la oficina de TECHO
capa_oficina = folium.FeatureGroup(name='Oficina')
folium.Circle(location=oficina, radius=distancia, color='black', weight=0.5).add_to(capa_oficina)
folium.Circle(location=oficina, radius=distancia/2, color='black', weight=0.5).add_to(capa_oficina)
folium.Circle(location=oficina, radius=10, color='black', fill=True, fill_opacity=1).add_to(capa_oficina)

## Creación de capas con información de interés del Censo 2020

In [ ]:
# Eliminar la información de las colonias no censadas
colonias = colonias[~(colonias == 'ND2020').any(axis=1)]

In [ ]:
# Lista para guardar las capas
capas_censo = []

# Por cada serie de datos de interés
for i in range(3, len(series)):

  # Identificar la serie y sus datos espaciales
  datos = colonias.iloc[:, [0, i, -1]].copy()
  datos[datos.columns[1]] = datos[datos.columns[1]].str.replace(',', '').astype(float)

  # Crear una capa con los datos de la serie
  capa = folium.Choropleth(
      geo_data = datos,
      data = datos,
      columns = datos.columns,
      key_on = f'feature.properties.{datos.columns[0]}',
      name = datos.columns[1],
      fill_color = 'YlGnBu',
      line_opacity = 0,
      overlay = False)

  # Borrar la leyenda
  for key in capa._children:
    if key.startswith('color_map'):
        del(capa._children[key])

  # Identificar información de interés
  datos = colonias.iloc[:, [1, 2, i, -1]]

  # Añadir información de interés para la serie
  folium.GeoJson(
    datos,
    style_function = lambda _: {'color': 'gray', 'fillColor':'transparent', 'weight': 0.1},
    tooltip = folium.GeoJsonTooltip(fields=datos.columns[:-1].tolist())
  ).add_to(capa)

  # Añadir la capa a la lista
  capas_censo.append(capa)

## Combinar todas las capas en un mapa

In [ ]:
# Crear un mapa centrado en la oficina de TECHO
mapa = folium.Map(location=oficina, zoom_start=11, tiles=None)

# Por cada capa con información de interés del Censo 2020
for capa in capas_censo:
  capa.add_to(mapa) # Añadir la capa al mapa

# Añadir la capa 'Oficina'
capa_oficina.add_to(mapa)

# Añadir mapa base
folium.TileLayer('cartodbpositron', name='Mapa', overlay=True).add_to(mapa)

# Añadir un controlador de capas
folium.LayerControl().add_to(mapa)

# Guardar el mapa como archivo HTML
mapa.save('/content/drive/MyDrive/TECHO/InformacionCenso2020/InformacionCenso2020.html')

# Visualizar el mapa
mapa

Output hidden; open in https://colab.research.google.com to view.